In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [10]:
import os
import numpy as np
import pandas as pd
import tensorflow as tf
from os.path import join
from tensorflow.keras.models import Model
from tensorflow.keras.layers import LSTM, Dense, Embedding, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import Adam
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

In [11]:
class BLEUCallback(tf.keras.callbacks.Callback):
    def __init__(self, X_val, Y_val, tokenizer):
        super().__init__()
        self.X_val = X_val
        self.Y_val = Y_val
        self.tokenizer = tokenizer
        self.smoothie = SmoothingFunction().method1  # Évite un BLEU = 0 sur des phrases courtes

    def on_epoch_end(self, epoch, logs=None):
        bleu_scores = []
        for i, motion in enumerate(self.X_val):
            pred_sentence = self.generate_text(motion)
            ref_sentence = self.tokenizer.sequences_to_texts([self.Y_val[i]])[0]

            bleu = sentence_bleu([ref_sentence.split()], pred_sentence.split(), smoothing_function=self.smoothie)
            bleu_scores.append(bleu)

            # Affichage à chaque prédiction
            print(f"🎯 Référence: {ref_sentence}")
            print(f"🤖 Prédiction: {pred_sentence}")
            print(f"🔵 BLEU Score: {bleu:.4f}\n{'-'*40}")

        avg_bleu = np.mean(bleu_scores)
        print(f"🔹 BLEU moyen après epoch {epoch+1}: {avg_bleu:.4f}")

    def generate_text(self, motion):
        input_seq = np.expand_dims(motion, axis=0)  # Ajouter une dimension pour batch

        # Créer un modèle d'encodeur pour obtenir state_h et state_c
        encoder_model = Model(inputs=self.model.input, outputs=[self.model.layers[1].output, self.model.layers[2].output])
        state_h, state_c = encoder_model.predict(input_seq)

        # Initialisation du séquenceur cible
        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = self.tokenizer.word_index['startseq']

        decoded_sentence = []
        for _ in range(20):  # max_len (limité ici à 20 mots)
            # Décoder la séquence en utilisant l'état de l'encodeur
            output_tokens, state_h, state_c = self.model.layers[3](target_seq, initial_state=[state_h, state_c])

            # Sélectionner le mot avec la plus grande probabilité
            word_idx = np.argmax(output_tokens[0, -1, :])
            word = self.tokenizer.index_word.get(word_idx, '')

            # Arrêter si l'on atteint 'endseq' ou un mot inconnu
            if word == 'endseq' or word == '':
                break

            decoded_sentence.append(word)
            target_seq[0, 0] = word_idx  # Mise à jour du séquenceur cible

        return ' '.join(decoded_sentence)

In [12]:
def load_data(file_list, motion_data_dir, text_data_dir):
    data = {}
    with open(file_list, 'r') as f:
        titles = f.read().splitlines()
        for t in titles:
            npy_file = f"{t}.npy"
            motion_data = np.load(join(motion_data_dir, npy_file))

            # Charger et traiter la description
            txt_file = f"{text_data_dir}{t}.txt"
            with open(txt_file, 'r', encoding='utf-8') as m:
                desc = m.readline().split('#')[0].capitalize()

            data[desc] = motion_data

    return pd.DataFrame({'Description': list(data.keys()), 'Motion': list(data.values())})

def pad_motion_sequences(motions, max_length):
    T = len(motions)
    N = motions[0].shape[1]  # N
    d = motions[0].shape[2]  # d

    # Initialisation du tenseur avec des zéros
    mtpadded = np.zeros((T, max_length, N, d))

    for i, motion in enumerate(motions):
        T = motion.shape[0]  # Longueur réelle de la séquence
        mtpadded[i, :T, :, :] = motion  # Copier la séquence dans le tenseur

    return mtpadded

In [13]:
# Répertoires des données
direct = '/kaggle/input/generative-ai-for-3-d-human-motion-description/HumanML3D - Challenge/'
motion_data_dir = "/kaggle/input/generative-ai-for-3-d-human-motion-description/HumanML3D - Challenge/motions/"
text_data_dir = "/kaggle/input/generative-ai-for-3-d-human-motion-description/HumanML3D - Challenge/texts/"

# Charger les ensembles de données
traindf = load_data(direct+'train.txt', motion_data_dir, text_data_dir)
valdf = load_data(direct+'val.txt', motion_data_dir, text_data_dir)

# Affichage des tailles des ensembles
print('Train:', traindf.shape)
print('Validation:', valdf.shape)

xtrain, ytrain = traindf['Motion'], traindf['Description']
xval, yval = valdf['Motion'], valdf['Description']

Train: (15433, 2)
Validation: (3592, 2)


In [14]:
# Configuration des paramètres
hidden_units = 256
emb_dim = 128
max_motion = max([x.shape[0] for x in xtrain] )

# Tokenizer
tknz = Tokenizer()
tknz.fit_on_texts(ytrain.tolist() + ['startseq', 'endseq'])
vocab_size = len(tknz.word_index) + 1

# Transformer les descriptions en vecteurs
ytrainseq = tknz.texts_to_sequences(ytrain)
yvalseq = tknz.texts_to_sequences(yval)
max_seq = max(len(seq) for seq in ytrainseq)

# Padding
xtrainpad = pad_motion_sequences(xtrain, max_motion)
xvalpad = pad_motion_sequences(xval, max_motion)

xtrainpad = np.reshape(xtrainpad, (xtrainpad.shape[0], xtrainpad.shape[1], -1))  # Forme (x, x, 66)
xvalpad = np.reshape(xvalpad, (xvalpad.shape[0], xvalpad.shape[1], -1))
print("Forme des données de mouvement (train) :", xtrainpad.shape)
print("Forme des données de mouvement (validation) :", xvalpad.shape)

ytrainpad = pad_sequences(ytrainseq, maxlen=max_seq)
yvalpad = pad_sequences(yvalseq, maxlen=max_seq)
print("Forme des données de texte (train) :", ytrainpad.shape)
print("Forme des données de texte (validation) :", yvalpad.shape)

Forme des données de mouvement (train) : (15433, 469, 66)
Forme des données de mouvement (validation) : (3592, 469, 66)
Forme des données de texte (train) : (15433, 85)
Forme des données de texte (validation) : (3592, 85)


In [15]:
# Encoder LSTM pour les poses 3D
motion_input = Input(shape=(max_motion, 22, 3))  # (nb_frames, features)
reshape = tf.keras.layers.Reshape((max_motion, 66))(motion_input)

encoder_lstm = LSTM(hidden_units, return_state=True)
_, state_h, state_c = encoder_lstm(reshape)
encoder_states = [state_h, state_c]

encoder_model = Model(motion_input, encoder_states)

# Décodeur pour le texte
text_input = Input(shape=(max_seq,))
embedding = Embedding(input_dim=vocab_size, output_dim=emb_dim, mask_zero=True)(text_input)
decoder_lstm = LSTM(hidden_units, return_sequences=True, return_state=True)
decoder_output, _, _ = decoder_lstm(embedding, initial_state=encoder_states)
decoder_dense = Dense(vocab_size, activation='softmax')
output = decoder_dense(decoder_output)

# Modèle final
model = Model([reshape, text_input], output)
model.compile(optimizer=Adam(0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

print('OK')

OK


In [17]:
# Entrées du modèle de décodage
decoder_input = Input(shape=(1,))  # Un seul token à la fois
decoder_state_input_h = Input(shape=(hidden_units,))
decoder_state_input_c = Input(shape=(hidden_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

# Embedding du token actuel
decoder_embedding = Embedding(input_dim=vocab_size, output_dim=emb_dim, mask_zero=True)(decoder_input)

# LSTM du décodeur (utilisation en inférence)
decoder_output, state_h, state_c = decoder_lstm(decoder_embedding, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]

# Couche de sortie
decoder_output = decoder_dense(decoder_output)

# Modèle du décodeur pour la prédiction
decoder_model = Model([decoder_input] + decoder_states_inputs, [decoder_output] + decoder_states)

print('OK')

OK


In [18]:
# Entraînement
model.fit([xtrainpad, ytrainpad], ytrainpad,
          batch_size=32, epochs=20)

Epoch 1/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 654s 1s/step - accuracy: 0.0446 - loss: 4.5389
Epoch 2/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 651s 1s/step - accuracy: 0.1349 - loss: 1.1275
Epoch 3/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 653s 1s/step - accuracy: 0.1468 - loss: 0.5617
Epoch 4/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 679s 1s/step - accuracy: 0.1521 - loss: 0.3406
Epoch 5/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 652s 1s/step - accuracy: 0.1540 - loss: 0.2325
Epoch 6/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 652s 1s/step - accuracy: 0.1550 - loss: 0.1609
Epoch 7/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 653s 1s/step - accuracy: 0.1552 - loss: 0.1111
Epoch 8/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 652s 1s/step - accuracy: 0.1576 - loss: 0.0725
Epoch 9/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 650s 1s/step - accuracy: 0.1580 - loss: 0.0453
Epoch 10/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 650s 1s/step - accuracy: 0.1579 - loss: 0.0280
Epoch 11/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 652s 1s/step - accuracy: 0.1572 - loss: 0.0157
Epoch 12/20
483/483 ━━━━━━━━━━━━━━━━━━━━ 

In [32]:
def temp_sample(output_tokens, temperature=1.0):
    # Applique une température sur les logits pour obtenir des probabilités plus "lisses"
    output_tokens = np.asarray(output_tokens).astype('float64')
    output_tokens = np.log(output_tokens + 1e-10) / temperature
    proba = np.exp(output_tokens) / np.sum(np.exp(output_tokens))
    
    # Utilisation de la méthode de sampling
    sampled_token_index = np.random.choice(len(proba[0, -1, :]), p=proba[0, -1, :])
    return sampled_token_index

In [34]:
for i in range(2):
    input_seq = pad_motion_sequences(np.expand_dims(xval[i], axis=0), max_motion)
    # Encoder : Extraire les états cachés
    input_seq = tf.convert_to_tensor(input_seq, dtype=tf.float32)
    states_value = encoder_model.predict(input_seq)  

    # Début de la séquence (token de départ)
    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = tknz.word_index['startseq']

    # Stockage de la phrase générée
    decoded_sentence = []

    for _ in range(max_seq-10):
        # Prédiction d'un mot
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Choisir le mot avec la plus grande probabilité
        # sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_token_index = temp_sample(output_tokens, temperature=0.8)
        sampled_word = tknz.index_word.get(sampled_token_index, '')

        # Arrêter si "endseq" est généré
        if sampled_word == "endseq":
            break

        decoded_sentence.append(sampled_word)

        # Mise à jour du token d'entrée
        # target_seq = np.zeros((1,1))
        target_seq[0, 0] = sampled_token_index

        # Mise à jour des états cachés du décodeur
        states_value = [h, c]

    print(f"Phrase prédite {i+1}: {' '.join(decoded_sentence)}")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 109ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 28ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 26ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
1/1 ━━━━━━━